<small><font color=gray>Notebook author: <a href="https://www.linkedin.com/in/olegmelnikov/" target="_blank">Oleg Melnikov</a> ©2021 onwards</font></small><hr style="margin:0;background-color:silver">

**[<font size=6>🧬Genomics</font>](https://www.kaggle.com/competitions/4dec23hse-genomics/rules)**. [**Instructions**](https://colab.research.google.com/drive/1owkYjuRGkx050LQnM3b3yTzd0Dr2XbeV) for running Colabs.

<small>**(Optional) CONSENT.** <mark>[ X ]</mark> We consent to sharing our Colab (after the assignment ends) with other students/instructors for educational purposes. We understand that sharing is optional and this decision will not affect our grade in any way. <font color=gray><i>(If ok with sharing your Colab for educational purposes, leave "X" in the check box.)</i></font></small>

In [ ]:
from google.colab import drive; drive.mount('/content/drive')   # OK to enable, if your kaggle.json is stored in Google Drive

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip -q install --upgrade --force-reinstall --no-deps kaggle > log  # upgrade kaggle package (to avoid a warning)
!mkdir -p ~/.kaggle                               # .kaggle folder must contain kaggle.json for kaggle executable to properly authenticate you to Kaggle.com
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json >log  # First, download kaggle.json from kaggle.com (in Account page) and place it in the root of mounted Google Drive
!cp kaggle.json ~/.kaggle/kaggle.json > log       # Alternative location of kaggle.json (without a connection to Google Drive)
!chmod 600 ~/.kaggle/kaggle.json                  # give only the owner full read/write access to kaggle.json
!kaggle config set -n competition -v 4dec23hse-genomics # set the competition context for the next few kaggle API calls. !kaggle config view - shows current settings
!kaggle competitions download >> log              # download competition dataset as a zip file
!unzip -o *.zip >> log                            # Kaggle dataset is copied as a single file and needs to be unzipped.
# !kaggle competitions leaderboard --show           # print public leaderboard

cp: cannot stat 'kaggle.json': No such file or directory
- competition is now set to: 4dec23hse-genomics


In [ ]:
!nvidia-smi --query-gpu=gpu_name,memory.total,memory.free,memory.used --format=csv # GPU specs

name, memory.total [MiB], memory.free [MiB], memory.used [MiB]
Tesla T4, 15360 MiB, 15101 MiB, 0 MiB


In [ ]:
%%time
%%capture
%reset -f
!pip -q install -U sentence-transformers > log
from IPython.core.interactiveshell import InteractiveShell as IS; IS.ast_node_interactivity = "all"
import numpy as np, pandas as pd, time, matplotlib.pyplot as plt, os, plotly
from sklearn.model_selection import train_test_split
from sentence_transformers import SentenceTransformer as SBERT
from sklearn.svm import SVC, LinearSVC, NuSVC
ToCSV = lambda df, fname: df.round(2).to_csv(f'{fname}.csv', index_label='id') # rounds values to 2 decimals

class Timer():
  def __init__(self, lim:'RunTimeLimit'=60*5): self.t0, self.lim, _ = time.time(), lim, print(f'⏳ started. You have {lim} sec. Good luck!')
  def ShowTime(self):
    msg = f'Runtime is {time.time()-self.t0:.0f} sec'
    print(f'\033[91m\033[1m' + msg + f' > {self.lim} sec limit!!!\033[0m' if (time.time()-self.t0-1) > self.lim else msg)

np.set_printoptions(linewidth=10000, precision=4, edgeitems=20, suppress=True)
pd.set_option('display.max_rows', 100, 'display.max_columns', 100, 'display.max_colwidth', 100, 'display.precision', 2, 'display.max_rows', 4)

CPU times: user 451 ms, sys: 12 ms, total: 463 ms
Wall time: 8.14 s


In [ ]:
vX = pd.read_csv('testX/testX.csv').set_index('id')
tYX = pd.read_csv('trainYX/trainYX.csv').set_index('id')
vX

,DNA
id,
100000,TTGATTAATAAGATTCCTTGACACCCTTTGTAAAGTTTCTATTTCGTGTGAAATATCTATCTCTTCAAATCCTTTTAATTTATCTAGGTATTTGCT...
100001,ATTAGTAACGGAGGATTTACTAGATGTTTGGATTTATATTCTAATTTTATTCAGGTGGAAGGGATTGTTTTATGATTCAATAGTATACAGAGAATA...
...,...
119998,CGTCGGCATGCTCGGGCAGTGCGGCGGGCCAGCAGCGTGCCAGTTGTCGCGGGGCGGCCGGGCATCGCGGCGCCGGGCGGCAGCACTCCCGCGAAG...
119999,GCGAGGGCACGAAGGCACGACGGCAACGGCGGCGAGGAGCGCTGTGGCAACCGTCTCCGCGTTTGCGTGCGTACAGCCGAGAGCTGGTTCGCGCAG...


In [ ]:
tmr = Timer() # runtime limit (in seconds). Add all of your code after the timer

⏳ started. You have 300 sec. Good luck!


❗Do not modify the setup above.

<hr color=red>

<font size=5>⏳</font> <strong><font color=orange size=5>Your Code, Documentation, Ideas and Timer - All Start Here...</font></strong>

Students: Keep all your definitions, code, documentation **between** ⏳ symbols.

## **Task 1. Preprocessing Pipeline**

Explain elements of your preprocessing pipeline i.e. feature engineering, subsampling, clustering, dimensionality reduction, etc.
1. Why did you choose these elements? (Something in EDA, prior experience,...? Btw, EDA is not required)
1. How do you evaluate the effectiveness of these elements?
1. What else have you tried that worked or didn't?

**Student's answer:**

To help understand the main objective of the task the proposed videos for learning about DNA were watched. It helped to understand that the bacterias can be classified based on the sequence of their DNA if the SBERT model is used in order to encode and combine chunks. You can observe the function "encode_and_combine" in the code which helps us to realise this idea.

There is also a commented function "parallel-embedding". However, it did not provide any additional improvements in the score. (might be that Collab has only one GPU available)

## **Task 2. Modeling Approach**
Explain your modeling approach, i.e. ideas you tried and why you thought they would be helpful.

1. How did these decisions guide you in modeling?
1. How do you evaluate the effectiveness of these elements?
1. What else have you tried that worked or didn't?

**Student's answer:**

It was decided to use the SBERT which is a Python framework for state-of-the-art sentence, text and image embeddings. It uses the BERT model combined with a siamese network which is a type of network architecture that contains two or more identical subnetworks used to generate feature vectors for each input and compare them. The version "paraphrase-MiniLM-L6-v2" was chosen.
There were also attempts to implement SBERT('all-mpnet-base-v2') which provided with 0.98 score but was extremely slow and SBERT('all-MiniLM-L6-v2') with the score of 0.965.
The train and test samples were divided into 95% and 5% from the given data respectively. It was done after seeing the idea of increasing the train set size.
For classification the Linear Support Vector classifier was chosen. You can see the parameters provided to it in the code.
There was also an attempt to use GridSearch and thus we chose an RBF kernel and C=2 with a 'scale' gamma.

[SBERT](https://www.sbert.net) generates 384-dimensional text embedding vectors for each text entry. See [more models](https://www.sbert.net/docs/pretrained_models.html).
* Only reputable publicly available embedding models are allowed (SBERT, USE, MUSE, LASER, ...). We want to prevent participants' training embeddings on test data.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import StandardScaler
from scipy.sparse import coo_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [ ]:
# data_slice = tYX.head(20000)

'''Google's model works really well'''
# tYXn = tYX.head(2000)

# import tensorflow_hub as hub
# !pip3 install tensorflow_text>=2.0.0rc0
# import numpy as np
# import tensorflow_text
# embed = hub.load("https://www.kaggle.com/models/google/universal-sentence-encoder/frameworks/TensorFlow2/variations/multilingual/versions/2")
# import numpy as np
# # Function to split DNA sequences into chunks

def split_string(s, length):
    return [s[i:i+length] for i in range(0, len(s), length)]

def kmer_split(sequence, k_length):
    return [sequence[i:i+k_length] for i in range(len(sequence) - k_length + 1)]

def pad_chunks(chunks, desired_length, padding_value):
    current_length = len(chunks)
    if current_length < desired_length:
        chunks += [padding_value] * (desired_length - current_length)
    return chunks


def encode_and_combine(dna_strings, chunk_length, padding_value):
    max_chunks = max(len(split_string(dna, chunk_length)) for dna in dna_strings)

    mean_embeddings = []
    concat_embeddings = []
    i = 0
    for dna in dna_strings:
        print(i / len(dna_strings))
        i += 1
        chunks = split_string(dna, chunk_length)
        # Pad chunks for concatenation
        padded_chunks = pad_chunks(chunks, max_chunks, padding_value)
        # Encode chunks using LASER
        chunk_embeddings = embed(chunks)
        # Calculate mean and concatenate embeddings
        mean_embedding = np.mean(chunk_embeddings, axis=0)
        concat_embedding = np.concatenate(chunk_embeddings, axis=0)
        mean_embeddings.append(mean_embedding)
        concat_embeddings.append(concat_embedding)

    return np.array(mean_embeddings), np.array(concat_embeddings)


def encode_dna_sequences(sequences_list):
    kmer_sequences = [' '.join(seq[i:i+3] for i in range(len(seq) - 2)) for seq in sequences_list]
    tfidf_encoder = TfidfVectorizer()
    encoded_vectors = tfidf_encoder.fit_transform(kmer_sequences)
    return encoded_vectors

def kmer_transform(dna_samples, k_dimensions):
    feature_collections = []
    for k in k_dimensions:
        kmer_series = [' '.join(kmer_split(seq, k)) for seq in dna_samples]
        cv = CountVectorizer()
        kmer_matrix = cv.fit_transform(kmer_series)
        feature_collections.append(kmer_matrix)
    merged_features = hstack(feature_collections)
    normalizer = StandardScaler(with_mean=False)
    return normalizer.fit_transform(merged_features)

k_dimensions = [4]

# temb2 = kmer_transform(tYX.head(20000)['DNA'], k_dimensions)
# vemb2 = kmer_transform(vX['DNA'], k_dimensions)

temb1 = encode_dna_sequences(tYX.head(100000)['DNA'])
vemb1 = encode_dna_sequences(vX['DNA'])

# temb3 = kmer_transform(tYX.head(20000)['DNA'], k_dimensions)
# vemb3 = kmer_transform(vX['DNA'], k_dimensions)

# chunk_length = 100  # Adjust as per LASER's token limit
# padding_value = '_' * chunk_length  # Replace with appropriate padding
# # Replace with your actual data
# # tMeanEmb2, tConcatEmb = encode_and_combine(tYXn.DNA, chunk_length, padding_value)
# # vMeanEmb2, vConcatEmb = encode_and_combine(vX.DNA, chunk_length, padding_value)

tmr.ShowTime()

"Google's model works really well"

Runtime is 37 sec


In [ ]:
temb1.shape
# temb2.shape

(100000, 64)

In [ ]:
tmr.ShowTime()
x_train_tEmb1, x_test_tEmb1, y_train, y_test = train_test_split(temb1, tYX.Y, random_state=21, train_size=0.7)
'''Unfortunately had to comment this out bcz it took too long ;( '''
# x_train_tEmb2, x_test_tEmb2, y_train2, y_test2 = train_test_split(temb2, tYX.Y.head(20000), random_state=21, train_size=0.99)
# x_train_tEmb3, x_test_tEmb3, y_train3, y_test3 = train_test_split(temb3, tYX.Y.tail(20000), random_state=21, train_size=0.99)

svc_model1 = SVC(random_state=1, C=2, gamma='scale', kernel='rbf')
svc_model1.fit(x_train_tEmb1, y_train)
# predictions1 = svc_model1.predict(x_test_tEmb1)
# accuracy1 = accuracy_score(y_test, predictions1)
tmr.ShowTime()

# svc_model2 = SVC(random_state=1)
# svc_model2.fit(x_train_tEmb2, y_train2)
# # predictions2 = svc_model2.predict(x_test_tEmb2)
# # accuracy2 = accuracy_score(y_test2, predictions2)
# tmr.ShowTime()

# svc_model3 = SVC(random_state=1)
# svc_model3.fit(x_train_tEmb3, y_train3)
# # predictions3 = svc_model2.predict(x_test_tEmb3)
# # accuracy3 = accuracy_score(y_test3, predictions3)
# tmr.ShowTime()

# # print("Accuracy of first model:", accuracy1)
# # print("Accuracy of second model:", accuracy2)
# # print("Accuracy of second model:", accuracy3)
# pred1 = svc_model1.predict(vemb1)
# pred2 = svc_model2.predict(vemb2)
# pred3 = svc_model3.predict(vemb3)

# import numpy as np
# from scipy.stats import mode

# combined_predictions = np.column_stack((pred1, pred2, pred3))
# final_predictions, _ = mode(combined_predictions, axis=1)
# final_predictions = final_predictions.flatten()


# pY = pd.DataFrame(final_predictions, index=vX.index, columns=['y'])   # predicted targets
# ToCSV((pY>0.5)*1, 'MySubmission_1')
# # pY = pd.DataFrame(svc_model2.predict(vemb2), index=vX.index, columns=['y'])   # predicted targets
# # ToCSV((pY>0.5)*1, 'MySubmission_2')

pY = pd.DataFrame(svc_model1.predict(vemb1), index=vX.index, columns=['y'])   # predicted targets
ToCSV((pY>0.5)*1, 'MySubmission_1')
tmr.ShowTime()

Runtime is 37 sec


'Unfortunately had to comment this out bcz it took too long ;( '

SVC(C=2, random_state=1)

Runtime is 170 sec
Runtime is 185 sec


In [ ]:
# from sklearn.model_selection import GridSearchCV
# from sklearn.svm import SVC

# # Define the parameter grid to search
# param_grid = {
#     'C': [0.1,0.5,1,2,10],  # Example values for the regularization parameter
#     'gamma': ['scale', 'auto'],  # Kernel coefficient
#     'kernel': ['linear', 'rbf'],  # Types of kernels to try
# }

# # Create an SVC model instance
# svc = SVC(random_state=7)

# # Create a GridSearchCV instance with the parameter grid
# grid_search = GridSearchCV(svc, param_grid, cv=3, scoring='accuracy', verbose=1)

# # Fit the GridSearchCV instance to your data
# grid_search.fit(temb1, tYX.Y)

# # Print the best parameters and the best score obtained
# print("Best Parameters:", grid_search.best_params_)
# print("Best Score:", grid_search.best_score_)

# # Evaluate on the training set (in-sample accuracy)
# in_sample_accuracy = grid_search.score(temb1, tYX.Y)
# print("In-sample Accuracy:", in_sample_accuracy)


# **References:**

Remember to cite your sources here as well! At the least, your textbook should be cited. Google Scholar allows you to effortlessly copy/paste an APA citation format for books and publications. Also cite StackOverflow, package documentation, and other meaningful internet resources to help your peers learn from these (and to avoid plagiarism claims).
1. https://www.statlearning.com/
2. https://www.youtube.com/results?search_query=nucleotides+genes+amino+acids+
3. https://www.sbert.net/
4. https://anirbansen2709.medium.com/sbert-how-to-use-sentence-embeddings-to-solve-real-world-problems-f950aa300c72

<font size=5>⌛</font> <strong><font color=orange size=5>Do not exceed competition's runtime limit!</font></strong>

<hr color=red>


In [ ]:
tmr.ShowTime()    # measure Colab's runtime. Do not remove. Keep as the last cell in your notebook.

Runtime is 185 sec


# 💡**Starter Ideas**

1. Learn about DNA [&#127910;](https://www.youtube.com/results?search_query=nucleotides+genes+amino+acids+)
1. Try a larger training sample.
1. Try longer training DNA strings, but SBERT may have a cap on string length, so you might split DNA into several strings and then concatenate or average resulting vectors
1. Try other pretrained SBERT models. Note that DNA sequence uses ACGT letters, but many other models were trained on multilingual text. So, you might prefer those that were trained on mostly ASCII.
1. SBERT is trained on word tokens (typically, separated by spaces), but DNA sequence has no spaces. Try placing spaces after every character or some semantically meaningful subsequences (this might require more domain knowledge).
1. Try Google's [USE](https://tfhub.dev/google/universal-sentence-encoder-multilingual/3) embedding models
1. Try Facebook's [LASER](https://github.com/facebookresearch/LASER) and [others](https://tfhub.dev/s?module-type=text-language-model).
1. Try [Enformer](https://tfhub.dev/deepmind/enformer/1) for gene expressions. See [DeepMind paper](https://deepmind.com/blog/article/enformer).
1. Try building your own embeddings on the given sequences. SBERT and other packages make it easy (just a few lines), but it may take too much time.
1. Assess distribution of character patterns (single, doubles, triplets, ...). For example, an ACGT string generates AC, CG, GT doubles and ACG and CGT triplets. Does one class have more subsequences of some type? This might be a feature in your model.
1. Try features built as counts of subsequences (singles, doubles, triplets, ...). Consider EDA first.
1. Concatenate or otherwise combine multiple embeddings derived from each gene string
1. Learn from [*The genetic code*](https://www.khanacademy.org/science/ap-biology/gene-expression-and-regulation/translation/a/the-genetic-code-discovery-and-properties), Khan Academy.
1. Learn from [*Apply Machine Learning Algorithms for Genomics Data Classification*](https://medium.com/mlearning-ai/apply-machine-learning-algorithms-for-genomics-data-classification-132972933723)
1. Learn from [*Efficient counting of k-mers in DNA sequences using a bloom filter*](https://bmcbioinformatics.biomedcentral.com/articles/10.1186/1471-2105-12-333) Páll Melsted et al. 2011
1. Try [Byte Pair Encoding](https://www.derczynski.com/papers/archive/BPE_Gage.pdf) and [SentencePiece](https://arxiv.org/pdf/1808.06226.pdf) to auto identification of "important" [k-mers](https://en.wikipedia.org/wiki/K-mer) (substrings)